In [1]:
import math
import numpy as np
import wandb

import torch
import torch_geometric
from torch_geometric.data import Data

from gnn_architectures import MyGnn

import gnn_io as gio
import gnn_architectures as garch

## 1. Define model and parameters

In [2]:
wandb.login()

# Define parameters 

num_epochs = 1000
project_name = 'pop_1pct_run'
train_ratio = 0.8

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

early_stopping = gio.EarlyStopping(patience=10, verbose=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: enatterer (tum-traffic-engineering). Use `wandb login --relogin` to force relogin


## 2. Load data

In [3]:
# Load the list of dictionaries
data_dict_list = torch.load('../data/dataset_1pct_0_100.pt')

# Reconstruct the Data objects
datalist = [Data(x=d['x'], edge_index=d['edge_index'], pos=d['pos'], y=d['y']) for d in data_dict_list]


In [4]:
# # Apply normalization to your dataset
dataset_normalized = gio.normalize_dataset(datalist)

baseline_error = gio.compute_baseline_error(datalist)
print(f'Baseline error: {baseline_error}')

Baseline error: 0.0058123222552239895


In [5]:
# Function to replace x with normalized_x and remove normalized_x
def replace_x_with_normalized_x(dataset):
    for data in dataset:
        if hasattr(data, 'normalized_x'):
            data.x = data.normalized_x
            del data.normalized_x
    return dataset

def cut_dimensions(dataset):
    for data in dataset:
        data.x = data.x[:, 0].view(-1, 1)  # Keep only the first column and reshape
    return dataset


# Apply the function to the dataset
dataset_updated = replace_x_with_normalized_x(dataset_normalized)

# Apply the function to the dataset
dataset_updated = cut_dimensions(dataset_updated)

## 3. Split into train and test set

In [6]:
train_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=True, batch_size=20, train_ratio=train_ratio)
valid_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=False, batch_size=20, train_ratio=train_ratio)

Total dataset length: 101
Training subset length: 80
Total dataset length: 101
Validation subset length: 20


## 4. Train the model

We first find a good model for one batch. 

In [7]:
def train(model, config=None, loss_fct=None, optimizer=None, train_dl=None, valid_dl=None, device=None, early_stopping=None):
    for epoch in range(config.epochs):
        model.train()
        for idx, data in enumerate(train_dl):
            input_node_features, targets = data.x.to(device), data.y.to(device)
            optimizer.zero_grad()

            # Forward pass
            predicted = model(data)
            train_loss = loss_fct(predicted, targets)
            
            # Backward pass
            train_loss.backward()
            optimizer.step()
            
            wandb.log({"train_loss": train_loss.item(), "epoch": epoch, "step": idx})
            # print(f"epoch: {epoch}, step: {idx}, loss: {train_loss.item()}")
        
        val_loss = garch.validate_model_pos_features(model, valid_dl, loss_fct, device)
        print(f"epoch: {epoch}, validation loss: {val_loss}")
        wandb.log({"loss": val_loss, "epoch": epoch})
            
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered. Stopping training.")
            break
    
    print("Best validation loss: ", val_loss)
    wandb.summary["val_loss"] = val_loss
    wandb.finish()
    return val_loss, epoch

In [8]:
batch_size = 16
output_layer_parameter = 'gcn'
hidden_size_parameter = 16
gat_layer_parameter = 0
gcn_layer_parameter = 0
lr = 0.001

wandb.login()

train_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=True, batch_size=batch_size, train_ratio=train_ratio)
valid_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=False, batch_size=batch_size, train_ratio=train_ratio)

wandb.init(
    project=project_name,
    config={
        "epochs": num_epochs,
        "batch_size": batch_size,
        "lr": lr,
        "early_stopping_patience": 10,
        "hidden_layer_size": hidden_size_parameter,
        "gat_layers": gat_layer_parameter,
        "gcn_layers": gcn_layer_parameter,
        "output_layer": output_layer_parameter,
        # "dropout": 0.15,
    }
)
config = wandb.config

gnn_instance = MyGnn(in_channels=3, out_channels=1, hidden_size=hidden_size_parameter, gat_layers=gat_layer_parameter, gcn_layers=gcn_layer_parameter, output_layer=output_layer_parameter)
model = gnn_instance.to(device)

best_val_loss, best_epoch = train(model, config=config, 
                                loss_fct=torch.nn.MSELoss(), 
                                optimizer=torch.optim.Adam(model.parameters(), lr=lr),
                                train_dl=train_dl, valid_dl=valid_dl,
                                device=device, early_stopping=early_stopping)

Total dataset length: 101
Training subset length: 80
Total dataset length: 101
Validation subset length: 16


Model initialized
MyGnn(
  (pointLayer): PointNetConv(local_nn=Sequential(
    (0): Linear(in_features=3, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
  ), global_nn=Sequential(
    (0): Linear(in_features=16, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=16, bias=True)
  ))
  (output_layer): GCNConv(16, 1)
)
epoch: 0, validation loss: 0.007236017379909754
epoch: 1, validation loss: 0.007401992566883564
EarlyStopping counter: 1 out of 10
epoch: 2, validation loss: 0.008257605135440826
EarlyStopping counter: 2 out of 10
epoch: 3, validation loss: 0.006101993843913078
epoch: 4, validation loss: 0.00604810006916523
epoch: 5, validation loss: 0.006332843564450741
EarlyStopping counter: 1 out of 10
epoch: 6, validation loss: 0.0058766501024365425
epoch: 7, validation loss: 0.005879996344447136
EarlyStopping counter: 1